In [18]:
import json
import utils
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset,DataLoader
import os

In [19]:
with open('trainer.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []

for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        word = utils.tokenizer(pattern)
        all_words.extend(word)
        xy.append((word,tag))

ignored_punct = ['!','?',',','.']

all_words = [utils.stemer(w) for w in all_words if w not in ignored_punct]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

In [20]:
X_train = []
Y_train = []

In [21]:
for (pattern_sentance,tag) in xy:
    bag = utils.bagger(pattern_sentance,all_words)
    X_train.append(bag)

    label = tags.index(tag)
    Y_train.append(label)

X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [22]:
class ChatDataset(Dataset):
    def __init__(self):
        super().__init__()

        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = Y_train

    def __getitem__(self,index):
        return self.x_data[index],self.y_data[index]
    
    def __len__(self):
        return self.n_samples

In [23]:
batch_size = 8
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,batch_size=batch_size,shuffle=True)

In [24]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [25]:
device

'cuda'

In [26]:
class ChatBot(nn.Module):
    def __init__(self,input_size,hidden_size,out_size):
        super().__init__()

        self.block = nn.Sequential(
            nn.Linear(in_features=input_size,out_features=hidden_size),
            nn.ReLU(),
            nn.Linear(in_features=hidden_size,out_features=hidden_size),
            nn.ReLU(),
            nn.Linear(in_features=hidden_size,out_features=hidden_size),
            nn.ReLU(),
            nn.Linear(in_features=hidden_size,out_features=out_size)
        )

    def forward(self,x):
        return self.block(x)

In [27]:
model = ChatBot(input_size=len(all_words),hidden_size=16,out_size=len(tags)).to(device)

In [28]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(),lr=0.001)

In [31]:
for epoch in range(5000):
    for (w,label) in train_loader:
        model.train()
        w,label = w.to(device),label.to(device,dtype = torch.long)

        output = model(w)

        loss = loss_fn(output,label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 100 == 0:
        print(f"Epoch: {epoch+1} | Loss: {loss.item()}")


print(f"Final Loss: {loss.item()}")

Epoch: 100 | Loss: 3.2123939990997314
Epoch: 200 | Loss: 3.3417797088623047
Epoch: 300 | Loss: 3.291473150253296
Epoch: 400 | Loss: 3.343209981918335
Epoch: 500 | Loss: 3.3902838230133057
Epoch: 600 | Loss: 3.249666929244995
Epoch: 700 | Loss: 3.257169246673584
Epoch: 800 | Loss: 3.3313708305358887
Epoch: 900 | Loss: 3.3542442321777344
Epoch: 1000 | Loss: 3.269580125808716
Epoch: 1100 | Loss: 3.2407915592193604
Epoch: 1200 | Loss: 3.198612928390503
Epoch: 1300 | Loss: 3.249332904815674
Epoch: 1400 | Loss: 3.1816141605377197
Epoch: 1500 | Loss: 3.339711904525757
Epoch: 1600 | Loss: 3.2775275707244873
Epoch: 1700 | Loss: 3.1676464080810547
Epoch: 1800 | Loss: 3.350839853286743
Epoch: 1900 | Loss: 3.302561044692993
Epoch: 2000 | Loss: 3.225886821746826
Epoch: 2100 | Loss: 3.3526012897491455
Epoch: 2200 | Loss: 3.2222487926483154
Epoch: 2300 | Loss: 3.2538540363311768
Epoch: 2400 | Loss: 3.215806245803833
Epoch: 2500 | Loss: 3.3843417167663574
Epoch: 2600 | Loss: 3.1933329105377197
Epoch: 

KeyboardInterrupt: 

In [30]:
import random
import json

import torch

from utils import bagger,tokenizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('trainer.json', 'r') as json_data:
    intents = json.load(json_data)

# FILE = "data.pth"
# data = torch.load(FILE)

# input_size = data["input_size"]
# hidden_size = data["hidden_size"]
# output_size = data["output_size"]
# all_words = data['all_words']
# tags = data['tags']
# model_state = data["model_state"]

model = ChatBot(input_size=len(all_words), hidden_size=16,out_size=len(tags)).to(device)
# model.load_state_dict(model_state)
# model.eval()

bot_name = "Sam"
print("Let's chat! (type 'quit' to exit)")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenizer(sentence)
    X = bagger(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")

Let's chat! (type 'quit' to exit)
Sam: I do not understand...
Sam: I do not understand...
